In [54]:
import pandas as pd
from os import path
from sklearn.preprocessing import OneHotEncoder

In [55]:
data_path = path.abspath("../data/US_Superstore_data.xls")
data = pd.read_excel(data_path)
# Country -> only have United States values
data = data.drop(columns=['Row ID', 'Order ID', 'Customer Name', 'Postal Code', 'Product ID', 'Product Name', 'Customer ID', 'Order Date', 'Ship Date', 'Country', "Region", "State"])
data

,Ship Mode,Segment,City,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,Henderson,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,Second Class,Consumer,Henderson,Furniture,Chairs,731.9400,3,0.00,219.5820
2,Second Class,Corporate,Los Angeles,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,Standard Class,Consumer,Fort Lauderdale,Furniture,Tables,957.5775,5,0.45,-383.0310
4,Standard Class,Consumer,Fort Lauderdale,Office Supplies,Storage,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,Miami,Furniture,Furnishings,25.2480,3,0.20,4.1028
9990,Standard Class,Consumer,Costa Mesa,Furniture,Furnishings,91.9600,2,0.00,15.6332
9991,Standard Class,Consumer,Costa Mesa,Technology,Phones,258.5760,2,0.20,19.3932
9992,Standard Class,Consumer,Costa Mesa,Office Supplies,Paper,29.6000,4,0.00,13.3200


In [56]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="distancias") # Replace "my_geopy_app" with a descriptive name for your application

# 39.828175 e 98.5795, centro do EUA
def distancia_do_centro(cidade:str):
  location = geolocator.geocode(cidade)
  if location:
        latitude = location.latitude
        longitude = location.longitude
        distancia_la = 39.828175 - latitude
        distancia_lo = 98.5795 - longitude
        distancia = (distancia_la**2 + distancia_lo**2)** 0.5
        
        return(distancia)
  
dict_coords = {}
for c in data["City"].unique():
    print(c)
    dict_coords[c] = distancia_do_centro(c)

data['City'] = data['City'].map(dict_coords)
data['City']=(data['City']-data['City'].mean())/data['City'].std()

Henderson
Los Angeles
Fort Lauderdale
Concord
Seattle
Fort Worth
Madison
West Jordan
San Francisco
Fremont
Philadelphia
Orem
Houston
Richardson
Naperville
Melbourne
Eagan
Westland
Dover
New Albany
New York City
Troy
Chicago
Gilbert
Springfield
Jackson
Memphis
Decatur
Durham
Columbia
Rochester
Minneapolis
Portland
Saint Paul
Aurora
Charlotte
Orland Park
Urbandale
Columbus
Bristol
Wilmington
Bloomington
Phoenix
Roseville
Independence
Pasadena
Newark
Franklin
Scottsdale
San Jose
Edmond
Carlsbad
San Antonio
Monroe
Fairfield
Grand Prairie
Redlands
Hamilton
Westfield
Akron
Denver
Dallas
Whittier
Saginaw
Medina
Dublin
Detroit
Tampa
Santa Clara
Lakeville
San Diego
Brentwood
Chapel Hill
Morristown
Cincinnati
Inglewood
Tamarac
Colorado Springs
Belleville
Taylor
Lakewood
Arlington
Arvada
Hackensack
Saint Petersburg
Long Beach
Hesperia
Murfreesboro
Layton
Austin
Lowell
Manchester
Harlingen
Tucson
Quincy
Pembroke Pines
Des Moines
Peoria
Las Vegas
Warwick
Miami
Huntington Beach
Richmond
Louisville
L

Normalização

In [57]:
data['Sales']=(data['Sales']-data['Sales'].mean())/data['Sales'].std()
data['Profit']=(data['Profit']-data['Profit'].mean())/data['Profit'].std()
data

,Ship Mode,Segment,City,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,Second Class,Consumer,0.168123,Furniture,Bookcases,0.051508,2,0.00,0.056590
1,Second Class,Consumer,0.168123,Furniture,Chairs,0.805593,3,0.00,0.815013
2,Second Class,Corporate,1.045221,Office Supplies,Labels,-0.345350,2,0.00,-0.092997
3,Standard Class,Consumer,-0.430383,Furniture,Tables,1.167630,5,0.45,-1.757397
4,Standard Class,Consumer,-0.430383,Office Supplies,Storage,-0.332919,2,0.20,-0.111587
...,...,...,...,...,...,...,...,...,...
9989,Second Class,Consumer,-0.427364,Furniture,Furnishings,-0.328298,3,0.20,-0.104816
9990,Standard Class,Consumer,1.032344,Furniture,Furnishings,-0.221258,2,0.00,-0.055595
9991,Standard Class,Consumer,1.032344,Technology,Phones,0.046078,2,0.20,-0.039544
9992,Standard Class,Consumer,1.032344,Office Supplies,Paper,-0.321315,4,0.00,-0.065470


 One-Hot Encoding

In [58]:
#encoder_states = OneHotEncoder(sparse_output=False)
#states = encoder_states.fit_transform(data[['State']])
#states_df = pd.DataFrame(states, columns=encoder_states.get_feature_names_out(['State']))

encoder_segment= OneHotEncoder(sparse_output=False)
segment = encoder_segment.fit_transform(data[['Segment']])
segment_df = pd.DataFrame(segment, columns=encoder_segment.get_feature_names_out(['Segment']))

encoder_ship_m= OneHotEncoder(sparse_output=False)
ship_m = encoder_ship_m.fit_transform(data[['Ship Mode']])
ship_df = pd.DataFrame(ship_m, columns=encoder_ship_m.get_feature_names_out(['Ship Mode']))

#encoder_region= OneHotEncoder(sparse_output=False)
#region = encoder_region.fit_transform(data[['Region']])
#region_df = pd.DataFrame(region, columns=encoder_region.get_feature_names_out(['Region']))

encoder_category= OneHotEncoder(sparse_output=False)
category = encoder_category.fit_transform(data[['Category']])
category_df = pd.DataFrame(category, columns=encoder_category.get_feature_names_out(['Category']))

#encoded_data = pd.concat([states_df, segment_df, ship_df, region_df, category_df, data["Sales"], data["Quantity"], data["Discount"], data["Profit"]], axis=1)
encoded_data = pd.concat([segment_df, ship_df, category_df, data["Sales"], data["Quantity"], data["Discount"], data["Profit"]], axis=1)
encoded_data

,Segment_Consumer,Segment_Corporate,Segment_Home Office,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,Category_Furniture,Category_Office Supplies,Category_Technology,Sales,Quantity,Discount,Profit
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.051508,2,0.00,0.056590
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.805593,3,0.00,0.815013
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-0.345350,2,0.00,-0.092997
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.167630,5,0.45,-1.757397
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.332919,2,0.20,-0.111587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-0.328298,3,0.20,-0.104816
9990,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-0.221258,2,0.00,-0.055595
9991,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.046078,2,0.20,-0.039544
9992,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.321315,4,0.00,-0.065470


In [59]:
data_path = path.abspath("../data/processed_data/US_Superstore_data.xls")
encoded_data.to_csv(str(data_path), index=False)